In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import xgboost as xgb
from preprocess import preprocess_data
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
pd.set_option('max_columns', None)

In [3]:
start = time.time()
df_train = pd.read_csv("raw_data/train_data.csv")
df_test = pd.read_csv("raw_data/test_data.csv")
end = time.time()
print("Data loaded in ", end - start, " seconds.")

Data loaded in  23.849340677261353  seconds.


In [4]:
df_train.head(1)

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,27,104.77,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [5]:
df_test.head(1)

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff
0,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,3180,3,4.5,1,2.94,0.0691,5.03,119.0,0,19222,1,10,2,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Preprocess Data

In [6]:
preprocessed_train = preprocess_data(df_train, split='train')
print('------')
preprocessed_test = preprocess_data(df_test, split='test')

------


In [7]:
preprocessed_train.head(1)

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,12,187,0.0,0.0,219,893,3,3.5,1,2.83,0.0438,4.95,27,104.77,0,23246,1,0,4,0,1,1,-330.0,1301.234406,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0


In [8]:
preprocessed_test.head(1)

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff
0,1,24,216,0.0,0.0,219,3180,3,4.5,1,2.94,0.0691,5.03,119.0,0,19222,1,10,2,0,1,0,-330.0,1312.801653,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
print(f'Size of Train data:{len(df_train)}')
print(f'Size of Test data:{len(df_test)}')

Size of Train data:4958347
Size of Test data:4959183


## Make Relevance Targets

Create a column called score which would be 
df['score'] = 4 * df.booking_bool + df.click_bool

In [15]:
preprocessed_train['rel'] = 4 * preprocessed_train.booking_bool + preprocessed_train.click_bool

In [16]:
X = preprocessed_train.drop(['rel', 'booking_bool', 'click_bool', 'gross_bookings_usd', 'position'], axis=1)

In [17]:
print(f'num_features:{X.shape[1]-1}') # -1 as we donot use srch_id
X.head(1)

num_features:48


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff
0,1,12,187,0.0,0.0,219,893,3,3.5,1,2.83,0.0438,4.95,104.77,0,23246,1,0,4,0,1,1,-330.0,1301.234406,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Train Test Split

In [19]:
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(test_size=.40, n_splits=1, random_state = 7).split(X, groups=X['srch_id'])
X_train_inds, X_val_inds = next(gss)

In [20]:
assert len(X) == len(X_train_inds) + len(X_val_inds)

In [21]:
train_data= X.iloc[X_train_inds]
X_train = train_data.loc[:, ~train_data.columns.isin(['srch_id','rel'])]
y_train = train_data.loc[:, train_data.columns.isin(['rel'])]
train_groups = train_data.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

In [24]:
val_data= X.iloc[X_val_inds]
#We need to keep the id for later predictions
X_val = val_data.loc[:, ~val_data.columns.isin(['srch_id','rel'])]
X_val_with_srch_id = val_data.loc[:, ~val_data.columns.isin(['rel'])]
y_val = val_data.loc[:, val_data.columns.isin(['rel'])]
eval_groups = X_val_with_srch_id.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

In [37]:
import xgboost as xgb

model = xgb.XGBRanker(  
    tree_method='gpu_hist',
    booster='gbtree',
    objective='rank:ndcg',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=10, 
    subsample=0.75,
    predictor='gpu_predictor'
    )

model.fit(X_train, y_train, group=train_groups, eval_set=[(X_val, y_val)], 
          eval_group = [eval_groups], eval_metric='ndcg', verbose=True)

[0]	validation_0-ndcg:1.00000
[1]	validation_0-ndcg:1.00000
[2]	validation_0-ndcg:1.00000
[3]	validation_0-ndcg:1.00000
[4]	validation_0-ndcg:1.00000
[5]	validation_0-ndcg:1.00000
[6]	validation_0-ndcg:1.00000
[7]	validation_0-ndcg:1.00000
[8]	validation_0-ndcg:1.00000
[9]	validation_0-ndcg:1.00000


XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=0.9, enable_categorical=False,
          eta=0.05, gamma=0, gpu_id=0, importance_type=None,
          interaction_constraints='', learning_rate=0.1, max_delta_step=0,
          max_depth=6, min_child_weight=1, missing=nan,
          monotone_constraints='()', n_estimators=10, n_jobs=16,
          num_parallel_tree=1, objective='rank:ndcg', predictor='gpu_predictor',
          random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
          subsample=0.75, tree_method='gpu_hist', validate_parameters=1,
          verbosity=None)

In [ ]:
X_train

In [30]:
model.feature_importances_.round(decimals=2)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [40]:
def recommend_properties(model, data):
    result = pd.DataFrame(columns=['srch_id', 'prop_id', 'scores'])
    grouped_data = data.groupby('srch_id')
    for group_name, group in tqdm(grouped_data):
        scores = model.predict(group.loc[:, ~group.columns.isin(['srch_id'])])
        sorted_group = sort_properties(group_name, scores, group['prop_id'])
        result = result.append(sorted_group)
    return result

In [41]:
def sort_properties(srch_id, scores, prop_ids):
    sorted_group = pd.DataFrame(columns=['srch_id', 'prop_id', 'scores'])
    sorted_group['srch_id'] = len(scores)*[srch_id]
    sorted_group['prop_id'] = prop_ids.values
    sorted_group['scores'] = scores
    sorted_group = sorted_group.sort_values(by='scores', ascending=False)
    return sorted_group 

In [42]:
start = time.time()
recommendations = recommend_properties(model, preprocessed_test)
end = time.time()
print(f'Total Time for validation prediction:{end - start}')

  0%|                                                             | 10/199549 [00:01<4:30:48, 12.28it/s]

prediction time:0.0060002803802490234
sort time:0.0030007362365722656
prediction time:0.00800180435180664
sort time:0.0030007362365722656
prediction time:0.007001161575317383
sort time:0.002000570297241211
prediction time:0.007001638412475586
sort time:0.0030007362365722656
prediction time:0.00800180435180664
sort time:0.002000570297241211
prediction time:0.007001638412475586
sort time:0.004001140594482422
prediction time:0.00600123405456543
sort time:0.0030007362365722656
prediction time:0.007001399993896484
sort time:0.002000570297241211
prediction time:0.007001638412475586
sort time:0.0030007362365722656
prediction time:0.00600123405456543
sort time:0.0030007362365722656
prediction time:0.00600123405456543
sort time:0.0030007362365722656
prediction time:0.00800180435180664
sort time:0.003000497817993164
prediction time:0.00600123405456543
sort time:0.0030007362365722656
prediction time:0.007001638412475586
sort time:0.003000497817993164
prediction time:0.00600123405456543
sort time:

  0%|                                                             | 29/199549 [00:01<1:30:19, 36.82it/s]


prediction time:0.00600123405456543
sort time:0.004001617431640625
prediction time:0.007001161575317383
sort time:0.0030007362365722656
prediction time:0.007001638412475586
sort time:0.003000497817993164
prediction time:0.00600123405456543
sort time:0.0030007362365722656
prediction time:0.006001710891723633
sort time:0.00400090217590332
prediction time:0.007001399993896484
sort time:0.003000974655151367
prediction time:0.007001399993896484
sort time:0.003000497817993164
prediction time:0.00600123405456543
sort time:0.004001617431640625
prediction time:0.00600123405456543
sort time:0.003000974655151367
prediction time:0.00800180435180664
sort time:0.004001140594482422
prediction time:0.006000995635986328
sort time:0.0040013790130615234
prediction time:0.00800180435180664
sort time:0.004000663757324219
prediction time:0.007001161575317383
sort time:0.003000974655151367
prediction time:0.00800180435180664
sort time:0.004001140594482422
prediction time:0.007001638412475586
sort time:0.003

  0%|                                                               | 47/199549 [00:01<58:57, 56.39it/s]

...

  4%|██▎                                                        | 7899/199549 [02:15<1:11:16, 44.81it/s]

sort time:0.004000663757324219
prediction time:0.008002042770385742
sort time:0.0030012130737304688
prediction time:0.009002208709716797
sort time:0.003000974655151367
prediction time:0.0070018768310546875
sort time:0.0030012130737304688
prediction time:0.00800180435180664
sort time:0.004000663757324219
prediction time:0.009001970291137695
sort time:0.0030007362365722656
prediction time:0.007001638412475586
sort time:0.003000497817993164
prediction time:0.009002447128295898
sort time:0.004001140594482422
prediction time:0.00900125503540039
sort time:0.003000974655151367
prediction time:0.0070018768310546875


  4%|██▍                                                          | 7910/199549 [02:16<54:55, 58.15it/s]

sort time:0.004000663757324219
prediction time:0.008002042770385742
sort time:0.004000663757324219
prediction time:0.008001565933227539
sort time:0.003000974655151367
prediction time:0.008002042770385742
sort time:0.0030012130737304688
prediction time:0.00800180435180664
sort time:0.0030007362365722656
prediction time:0.0070018768310546875
sort time:0.00400090217590332
prediction time:0.008002042770385742
sort time:0.004000663757324219


KeyboardInterrupt: 

In [ ]:
X_test